# ADS 509 Assignment 2.1: Tokenization, Normalization, Descriptive Statistics 

This notebook holds Assignment 2.1 for Module 2 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In the previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we explore some of the textual features of those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 

This assignment asks you to write a short function to calculate some descriptive statistics on a piece of text. Then you are asked to find some interesting and unique statistics on your corpora. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english")

In [2]:
# Add any additional import statements you need here
from lexical_diversity import lex_div as ld
import os.path
import datetime

In [3]:
# change `data_location` to the location of the folder on your machine.
data_location = "D:/USD-ADS_graduate/ADS-509/Module 1/ADS_509_module1/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

In [4]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    num_tokens = 13
    num_unique_tokens = 9
    lexical_diversity = 0.692
    num_characters = 55
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    

In [5]:
text = """here is some example text with other example text here in this text""".split()
assert(descriptive_stats(text, verbose=True)[0] == 13)
assert(descriptive_stats(text, verbose=False)[1] == 9)
assert(abs(descriptive_stats(text, verbose=False)[2] - 0.69) < 0.02)
assert(descriptive_stats(text, verbose=False)[3] == 55)


There are 13 tokens in the data.
There are 9 unique tokens in the data.
There are 55 characters in the data.
The lexical diversity is 0.692 in the data.


Q: Why is it beneficial to use assertion statements in your code? 

A: To ensure that our code/logic and texts/strings are correct before moving to the next step.

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [6]:
# Read in the lyrics data
lyrics_path = data_location + lyrics_folder
artists = os.listdir(lyrics_path)

a = []
b = []
c = []

for artist in artists:
    raw_song = os.listdir(lyrics_path+artist)
    for song in raw_song:
        a.append(str(artist))
        f = open(lyrics_path+artist+'/'+song, "r")
        song = re.search('_(.+?).txt',song).group(1)
        b.append(song)  
        c.append(f.read())

lyrics_df = pd.DataFrame({'artist':a,
                           'song':b,
                           'song_content':c})

lyrics_df.head()

,artist,song,song_content
0,Mariah Carey,allinyourmind,['allinyourmind']\n\nCome closer\nYou seem so ...
1,Mariah Carey,aloneinlove,['aloneinlove']\n\nSwept me away\nBut now I'm ...
2,Mariah Carey,andyoudontremember,['andyoudontremember']\n\nShattered dreams\nCu...
3,Mariah Carey,anytimeyouneedafriend,['anytimeyouneedafriend']\n\nIf you're lonely\...
4,Mariah Carey,cantletgo,"['cantletgo']\n\nThere you are, holding her ha..."


In [7]:
# Read in the twitter data
twitter_path = data_location + twitter_folder
twitter_files = os.listdir(twitter_path)

mariah_df = pd.DataFrame()
nicki_df = pd.DataFrame()

mariah_des = pd.read_csv(os.path.join(twitter_path, 'MariahCarey_followers.txt'),sep='\t')['description']
nicki_des = pd.read_csv(os.path.join(twitter_path, 'NICKIMINAJ_followers.txt'),sep='\t')['description']

mariah_df['artist'] = ['Mariah Carey']*len(mariah_des)
mariah_df['description'] = mariah_des

nicki_df['artist'] = ['Nicki Minaj']*len(nicki_des)
nicki_df['description'] = nicki_des

twitter_df = pd.concat([mariah_df,nicki_df])
twitter_df['description'] = twitter_df['description'].astype(str)

## Data Cleaning

Now clean and tokenize your data. Remove punctuation chacters (available in the `punctuation` object in the `string` library), split on whitespace, fold to lowercase, and remove stopwords. Store your cleaned data, which must be accessible as an interable for `descriptive_stats`, in new objects or in new columns in your data frame. 



In [8]:
punctuation = set(punctuation) # speeds up comparison
punctuation.add('’')

In [9]:
# create your clean twitter data here

# remove punctuation
def remove_punctuations(text):
    for punc in punctuation:
        text = text.replace(punc, '')
    return text

# split_on_whitespace
def split_on_whitespace(text):
    text = text.split()
    return text

# fold to lower case
def fold_to_lower_case(text_list):
    text_list = [i.casefold() for i in text_list]
    return text_list

# remove stop words
def remove_stop_words(text_list):
    text_list = [i for i in text_list if not i in sw]
    return text_list



twitter_df["description_clean"] = twitter_df['description']

twitter_df["description_clean"] = twitter_df['description_clean'].apply(remove_punctuations)\
.apply(split_on_whitespace).apply(fold_to_lower_case).apply(remove_stop_words)

In [10]:
twitter_df.head()

,artist,description,description_clean
0,Mariah Carey,nan,[nan]
1,Mariah Carey,nan,[nan]
2,Mariah Carey,nan,[nan]
3,Mariah Carey,There’s nothing more inspiring than a story th...,"[theres, nothing, inspiring, story, touches, h..."
4,Mariah Carey,♑️ | Bi | Take It Or Leave It 🤷🏾‍♂️| Don’t sta...,"[♑️, bi, take, leave, 🤷🏾‍♂️, dont, start, nun,..."


In [11]:
# create your clean lyrics data here
lyrics_df['song_content_clean'] = lyrics_df['song_content']
lyrics_df['song_content_clean'] = lyrics_df['song_content_clean'].apply(remove_punctuations)\
.apply(split_on_whitespace).apply(fold_to_lower_case).apply(remove_stop_words)

In [12]:
lyrics_df.head()

,artist,song,song_content,song_content_clean
0,Mariah Carey,allinyourmind,['allinyourmind']\n\nCome closer\nYou seem so ...,"[allinyourmind, come, closer, seem, far, away,..."
1,Mariah Carey,aloneinlove,['aloneinlove']\n\nSwept me away\nBut now I'm ...,"[aloneinlove, swept, away, im, lost, dark, set..."
2,Mariah Carey,andyoudontremember,['andyoudontremember']\n\nShattered dreams\nCu...,"[andyoudontremember, shattered, dreams, cut, m..."
3,Mariah Carey,anytimeyouneedafriend,['anytimeyouneedafriend']\n\nIf you're lonely\...,"[anytimeyouneedafriend, youre, lonely, need, f..."
4,Mariah Carey,cantletgo,"['cantletgo']\n\nThere you are, holding her ha...","[cantletgo, holding, hand, lost, dying, unders..."


## Basic Descriptive Statistics

Call your `descriptive_stats` function on both your lyrics data and your twitter data and for both artists (four total calls). 

In [13]:
def descriptive_stats_ddy(tokens, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = ld.ttr(tokens)
    num_characters = sum([len(token) for token in tokens])
    top5_tokens = Counter(tokens).most_common()[:5]
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"These are the five most common tokens in the data: {top5_tokens}.")
    
        # print the five most common tokens
        
    #return([num_tokens, num_unique_tokens,
            #lexical_diversity,
            #num_characters])

In [14]:
# calls to descriptive_stats here
start_time = datetime.datetime.now()
mariah_lyrics  = lyrics_df['song_content_clean'].loc[lyrics_df['artist'] == 'Mariah Carey']
nicki_lyrics   = lyrics_df['song_content_clean'].loc[lyrics_df['artist'] == 'Nicki Minaj']
mariah_twitter = twitter_df['description_clean'].loc[twitter_df['artist'] == 'Mariah Carey']
nicki_twitter  = twitter_df['description_clean'].loc[twitter_df['artist'] == 'Nicki Minaj']

mariah_lyrics_sum = []
for i in range(len(mariah_lyrics)):
    mariah_lyrics_sum = mariah_lyrics_sum + mariah_lyrics[i]

nicki_lyrics_sum = []
for i in range(len(mariah_lyrics)):
    nicki_lyrics_sum = nicki_lyrics_sum + nicki_lyrics[i+len(mariah_lyrics)] # index for nicki is from 26 where mariah left over

mariah_twitter_sum = []
for i in range(len(mariah_twitter)):
    mariah_twitter_sum = mariah_twitter_sum + mariah_twitter[i]
    
nicki_twitter_sum = []
for i in range(len(nicki_twitter)):
    nicki_twitter_sum = nicki_twitter_sum + nicki_twitter[i]
    
print("For Mariah Carey's lyrics: \n")
print(descriptive_stats_ddy(mariah_lyrics_sum, verbose=True))
print('\n\n')
print("For Nicki Minaj's lyrics: \n")
print(descriptive_stats_ddy(nicki_lyrics_sum, verbose=True))
print('\n\n')
print("For Mariah Carey's followers' description: \n")
print(descriptive_stats_ddy(mariah_twitter_sum, verbose=True))
print('\n\n')
print("For Nicki Minaj's followers' description: \n")
print(descriptive_stats_ddy(nicki_twitter_sum, verbose=True))
print('\n\n')
end_time = datetime.datetime.now()
print("Total time consumed:", end_time-start_time)

For Mariah Carey's lyrics: /n
There are 3791 tokens in the data.
There are 681 unique tokens in the data.
The lexical diversity is 0.180 in the data.
There are 18510 characters in the data.
These are the five most common tokens in the data: [('dont', 117), ('love', 108), ('youre', 89), ('know', 76), ('baby', 71)].
[3791, 681, 0.17963597995251912, 18510]



For Nicki Minaj's lyrics: /n
There are 6626 tokens in the data.
There are 1983 unique tokens in the data.
The lexical diversity is 0.299 in the data.
There are 30778 characters in the data.
These are the five most common tokens in the data: [('im', 227), ('like', 150), ('get', 117), ('buck', 106), ('dont', 90)].
[6626, 1983, 0.2992755810443707, 30778]



For Mariah Carey's followers' description: /n
There are 337067 tokens in the data.
There are 77148 unique tokens in the data.
The lexical diversity is 0.229 in the data.
There are 1795419 characters in the data.
These are the five most common tokens in the data: [('nan', 49052), ('lo

Q: How do you think the "top 5 words" would be different if we left stopwords in the data? 

A: 

---

Q: What were your prior beliefs about the lexical diversity between the artists? Does the difference (or lack thereof) in lexical diversity between the artists conform to your prior beliefs? 

A: 



## Specialty Statistics

The descriptive statistics we have calculated are quite generic. You will now calculate a handful of statistics tailored to these data.

1. Ten most common emojis by artist in the twitter descriptions.
1. Ten most common hashtags by artist in the twitter descriptions.
1. Five most common words in song titles by artist. 
1. For each artist, a histogram of song lengths (in terms of number of tokens) 

We can use the `emoji` library to help us identify emojis and you have been given a function to help you.


In [17]:
def is_emoji(s):
    return(s in emoji.UNICODE_EMOJI['en'])

assert(is_emoji("❤️"))
assert(not is_emoji(":-)"))

AttributeError: module 'emoji' has no attribute 'UNICODE_EMOJI'

### Emojis 😁

What are the ten most common emojis by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Hashtags

What are the ten most common hashtags by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Song Titles

What are the five most common words in song titles by artist? The song titles should be on the first line of the lyrics pages, so if you have kept the raw file contents around, you will not need to re-read the data.


In [ ]:
# Your code here

### Song Lengths

For each artist, a histogram of song lengths (in terms of number of tokens). If you put the song lengths in a data frame with an artist column, matplotlib will make the plotting quite easy. An example is given to help you out. 


In [ ]:
num_replicates = 1000

df = pd.DataFrame({
    "artist" : ['Artist 1'] * num_replicates + ['Artist 2']*num_replicates,
    "length" : np.concatenate((np.random.poisson(125,num_replicates),np.random.poisson(150,num_replicates)))
})

df.groupby('artist')['length'].plot(kind="hist",density=True,alpha=0.5,legend=True)

Since the lyrics may be stored with carriage returns or tabs, it may be useful to have a function that can collapse whitespace, using regular expressions, and be used for splitting. 

Q: What does the regular expression `'\s+'` match on? 

A: 


In [ ]:
collapse_whitespace = re.compile(r'\s+')

def tokenize_lyrics(lyric) : 
    """strip and split on whitespace"""
    return([item.lower() for item in collapse_whitespace.split(lyric)])

In [ ]:
# Your lyric length comparison chart here. 